In [6]:
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Drugs table

In [26]:
drugs = pd.read_excel('bcdrugs.xlsx')
drugs.shape

(1203, 36)

Drugs columns = 'Drug ID', 'Drug Name', 'CAS Number', 'Chemical Name',
       'New Molecular Entity', 'Drug Interactions', 'Comparative Efficacy',
       'Highest Development Phase', 'Indication', 'Molecular Formula',
       'Mechanism Of Action', 'Organisation',
       'Pharmacodynamic Characteristics', 'Route Of Administration',
       'Route Of Elimination', 'Pharmacokinetic Characteristics',
       'Adverse Events', 'Therapeutic Class (EphMRA)',
       'Therapeutic Class (WHO)', 'Synonyms/Brand Names',
       'Antimicrobial Activity', 'Immunogenicity',
       'Last Significant Update Date', 'Therapeutic Area',
       'Orphan Designation', 'License Availability', 'Brands', 'On Fastrack',
       'Fastrack country', 'On PRIME', 'PRIME country', 'Biomarkers-Inclusion',
       'Biomarkers-Exclusion', 'Biomarkers-Outcome Measure',
       'Biomarkers-Safety', 'Biomarkers-not specified'

In [81]:
drugs['Highest Development Phase'].value_counts()

Marketed                                     175
Preclinical                                  167
Phase II                                     149
No development reported (Preclinical)         87
Phase III                                     84
Discontinued (II)                             84
Phase I                                       65
Discontinued (Preclinical)                    60
Discontinued (III)                            46
Phase I/II                                    44
Research                                      35
Discontinued (I)                              25
No development reported (Research)            21
No development reported (I)                   21
No development reported (II)                  20
Registered                                    15
Phase II/III                                  14
Discontinued (I/II)                           11
Discontinued (Preregistration)                10
No development reported (Clinical)            10
Preregistration     

## Trials table

In [7]:
trials = pd.read_excel('bcdrugsct.xlsx')
trials.shape

(4529, 41)

In [159]:
cols = ['Trial ID','Mechanism of action', 'Drug class (therapeutic effect)', 'Drug class (chemical)', \
        'Indication', 'Organisations', 'Trial Design', 'Location', 'Phase of Trial', 'Subject Age',\
        'Planned Subject Number', 'Trial Centre Details', 'Lead Centre', 'Trial Initiation date', \
        'Trial End date', 'Trial Status', 'Trial History', 'Diseases treated', 'Primary Drugs']
smaller_trials = trials[cols]
date_cols = ['Phase of Trial','Trial Initiation date', 'Trial End date', 'trial_start','trial_end']

In [133]:
df['Trial Initiation date'].head()

1      01 Dec 2013 (actual)
10     01 Dec 2015 (actual)
16     12 Feb 2019 (actual)
478    06 Dec 2018 (actual)
481    01 Feb 2014 (actual)
Name: Trial Initiation date, dtype: object

In [ ]:

def phase_1(drug_name):
    '''
    take earliest Phase I date and latest Phase II
    '''
    tucatinib_trials = trials[trials['Primary Drugs']==drug_name]
    trial_phases = ['Phase I', 'Phase II', 'Phase III']
    cleaned_tucatinib_trials = tucatinib_trials[tucatinib_trials['Phase of Trial'].isin(trial_phases)]
    phase_dummies = pd.get_dummies(cleaned_tucatinib_trials['Phase of Trial'])
    cleaned_tucatinib_trials_dummies = cleaned_tucatinib_trials.join(phase_dummies)
    #cleaned_tucatinib_trials_dummies[['Trial Initiation date', 'Trial End date']]
    df = cleaned_tucatinib_trials.join(phase_dummies)

    #not DRY as the answers to make multiargument apply functions involved more complicated things than I wanted.
    def start_to_datetime(x):
        date_str = re.findall('[\d\s\w]*', x['Trial Initiation date'])[0].strip()
        if date_str == '':
            return False
        else:
            return pd.to_datetime(date_str)

    def end_to_datetime(x):
        date_str = re.findall('[\d\s\w]*', x['Trial End date'])[0].strip()
        if date_str == '':
            return False
        else:
            return pd.to_datetime(date_str)

    df['trial_start'] = df.apply(start_to_datetime, axis=1)
    df['trial_end'] = df.apply(end_to_datetime, axis=1)

    t_phase_1_min = min(list(df[(df['trial_start']!=False) & (df['Phase of Trial']=='Phase I')]['trial_start'].values))
    t_phase_1_max = max(list(df[(df['trial_start']!=False) & (df['Phase of Trial']=='Phase I')]['trial_start'].values))

    return f'min is {t_phase_1_min} and max is {t_phase_1_max}'

phase_1('Xentuzumab')


# df['split_start_date'] = df['Trial Initiation date'].apply(lambda x: x.split(' '))
# df['split_end_date'] = df['Trial End date'].apply(lambda x: x.split(' '))
# df2 = df[df['split_start_date'].apply(lambda x: len(x[:3]))==3]
# df3 = df2[df2['split_end_date'].apply(lambda x: len(x[:3]))==3]
# df3[date_cols]
#df3.apply(lambda x: ' '.join(x['split_start_date']))
# in_progress_trials = trials[trials['split_start_date'].apply(lambda x: len(x[:3]))==3]
# planned_trials = trials[trials['split_start_date'].apply(lambda x: len(x[:3]))==2]
# # one_split = trials['split_end_date'].values[0]
# # pd.to_datetime(' '.join(one_split[:3]))







In [52]:
phase_two = smaller_trials[smaller_trials['Phase of Trial'] == 'Phase II']
phase_three = smaller_trials[smaller_trials['Phase of Trial'] == 'Phase III']

In [63]:
unique_drugs =list(smaller_trials['Primary Drugs'].unique())

In [185]:
unique_drugs

['AB-928',
 'Tucatinib',
 'Lenvatinib, Pembrolizumab',
 'Bevacizumab, Paclitaxel',
 'Xentuzumab',
 'Pembrolizumab',
 'Taselisib',
 'Adavosertib, Afatinib, AZD 4547, Binimetinib, Capivasertib, Crizotinib, Dabrafenib, Dasatinib, Defactinib, GSK 2636771, Larotrectinib, Nivolumab, Osimertinib, Palbociclib, Pertuzumab, Sapanisertib, Sunitinib, Taselisib, Trametinib, Trastuzumab, Trastuzumab emtansine, Vismodegib',
 'Anastrozole, Exemestane, Fulvestrant, Letrozole, Palbociclib',
 'Trastuzumab emtansine',
 'Pertuzumab',
 'KN 035',
 'Gallium 68 satoreotide, Lutetium-177-DOTA-satoreotide',
 'Avelumab, TRX 518',
 'Varlitinib',
 'Capecitabine, Tesetaxel',
 'Fulvestrant',
 'Afatinib',
 'Celecoxib',
 'Nivolumab, Trastuzumab-deruxtecan',
 'Abiraterone acetate',
 'Exemestane',
 'AZD 4547',
 'Margetuximab',
 'Capivasertib',
 'Alpelisib, Fulvestrant, Letrozole',
 'Eribulin',
 'Bevacizumab',
 'Cyclophosphamide, Pertuzumab, Trastuzumab',
 'Autologous cytotoxic T cell therapeutic Marker Cell Therapeutics'

In [79]:
trials[trials['Primary Drugs']=='Tucatinib']

,Trial ID,Trial Title,Mechanism of action,Drug class (therapeutic effect),Drug class (chemical),Drug class (biological),Indication,Organisations,Trial Design,Trial Endpoints,...,Trial End date,Trial Status,Trial History,Diseases treated,Biomarkers-Inclusion,Biomarkers-Exclusion,Biomarkers-Outcome Measure,Biomarkers-Safety,Biomarkers-not specified,Primary Drugs
1,700240183,"A Phase 1b, Open-label Study to Assess the Saf...","Antibody-dependent cell cytotoxicity, Antimeta...","Antineoplastics, Antineoplastics, Antineoplastics","2 ring heterocyclic compounds, Diamines, Furan...",Monoclonal antibodies,"Advanced breast cancer, Male breast cancer","Cascadian Therapeutics, Cascadian Therapeutics...","multicentre, open, parallel, prospective",Incidence of adverse events\ntime_frame: Up to...,...,01 Jul 2019 (planned),"Active, no longer recruiting",2019-03-06|Other trial event|Last checked agai...,Advanced breast cancer|treatment|second-line o...,HER2,HER2,NaN,NaN,NaN,Tucatinib
10,700257487,"Phase 2 Randomized, Double-Blinded, Controlled...","Antibody-dependent cell cytotoxicity, Antibody...","Antineoplastics, Antineoplastics, Antineoplast...","2 ring heterocyclic compounds, Diamines, Furan...","Monoclonal antibodies, Monoclonal antibodies","Advanced breast cancer, Carcinoma, Male breast...","Cascadian Therapeutics, Cascadian Therapeutics...","double-blind, multicentre, parallel, prospecti...",Progression-free survival (PFS) per RECIST 1.1...,...,01 Jul 2021 (planned),Recruiting,2019-03-06|Other trial event|Last checked agai...,Advanced breast cancer|treatment|second-line o...,HER2,HER2,NaN,NaN,NaN,Tucatinib
16,700304216,"A Phase 1, Open-Label, Fixed-Sequence, Drug-Dr...","AMP activated protein kinase stimulants, Emiss...","Antineoplastics, Antihyperglycaemics","2 ring heterocyclic compounds, Diamines, Furan...",NaN,"Breast cancer, Colorectal cancer","Seattle Genetics, Seattle Genetics","open, prospective",Maximum observed plasma concentration (Cmax) o...,...,01 Mar 2019 (planned),Recruiting,2019-03-06|Other trial event|Last checked agai...,Breast cancer|treatment|; Colorectal cancer|tr...,NaN,NaN,NaN,NaN,NaN,Tucatinib
478,700302859,"A Phase 1, Randomized, Partially Double-Blind,...","DNA gyrase inhibitors, ERBB 2 receptor antagon...","Antineoplastics, Antibacterials","2 ring heterocyclic compounds, Diamines, Furan...",NaN,"Breast cancer, Colorectal cancer","Seattle Genetics, Seattle Genetics","crossover, double-blind, prospective, randomised",Placebo-corrected change-from-baseline in QTcF...,...,01 Mar 2019 (planned),Recruiting,2019-02-03|Other trial event|Last checked agai...,Breast cancer|treatment|; Colorectal cancer|tr...,NaN,NaN,NaN,NaN,NaN,Tucatinib
481,700238859,"A Phase 1b, Open-label Study to Assess the Saf...","Apoptosis stimulants, ERBB 2 receptor antagoni...","Antineoplastics, Antineoplastics","2 ring heterocyclic compounds, Diamines, Furan...","Immunoconjugates, Immunotoxins, Monoclonal ant...","Advanced breast cancer, Male breast cancer","Cascadian Therapeutics, Cascadian Therapeutics...","multicentre, open, prospective",Incidence and severity of adverse events\nsafe...,...,01 Jul 2019 (planned),"Active, no longer recruiting",2019-02-01|Other trial event|Last checked agai...,Advanced breast cancer|treatment|stage IV; Mal...,HER2,HER2,NaN,NaN,NaN,Tucatinib
489,700301177,"An Open-label, Non-randomized, Single-dose, Pa...",ERBB 2 receptor antagonists,Antineoplastics,"2 ring heterocyclic compounds, Diamines, Furan...",NaN,"Breast cancer, Colorectal cancer","Seattle Genetics, Seattle Genetics","multicentre, open, parallel, prospective",Maximum observed concentration\ndescription: P...,...,01 May 2019 (planned),Recruiting,2019-02-01|Other trial event|Last checked agai...,Breast cancer|treatment|; Colorectal cancer|tr...,NaN,NaN,NaN,NaN,NaN,Tucatinib
570,700301239,"A Phase 1, Open-Label, Fixed-sequence, 5-part,...","14-alpha demethylase inhibitors, Angiogenesis ...","Antineoplastics, Antihyperlipidaemics, Antifun...","2 ring he

In [37]:
def trial_history_to_table(th):
    rows = str.split(th, ';')
    #columns_trial_1 = [str.split(i, '|') for i in rows_trial_1]
    table = [str.split(row, '|') for row in rows]
    return pd.DataFrame(table,columns=['Date','Status','Notes'])

trial_history_to_table(phase_three['Trial History'].values[0])

,Date,Status,Notes
0,2019-03-06,Other trial event,Last checked against ClinicalTrials.gov record.
1,2019-01-25,Other trial event,Last checked against the European Clinical Tri...
2,2018-12-26,Other trial event,New source identified and integrated (European...
3,2018-12-19,Status change - recruiting,Status changed from not yet recruiting to recr...
4,2018-11-06,New trial record,New trial record


In [18]:
pd.DataFrame([[1,2,3],[4,5,6]],columns=['a','b','c'])

,a,b,c
0,1,2,3
1,4,5,6


In [67]:
trials.head()

,Trial ID,Trial Title,Mechanism of action,Drug class (therapeutic effect),Drug class (chemical),Drug class (biological),Indication,Organisations,Trial Design,Trial Endpoints,...,Trial Status,Trial History,Diseases treated,Biomarkers-Inclusion,Biomarkers-Exclusion,Biomarkers-Outcome Measure,Biomarkers-Safety,Biomarkers-not specified,Primary Drugs,split_end_date
0,700301099,A Phase 1/1b Study to Evaluate the Safety and ...,"Adenosine A2A receptor antagonists, Adenosine ...","Antineoplastics, Antineoplastics, Antivirals, ...","Small molecules, Anthracyclines, Doxorubicins,...",NaN,"Advanced breast cancer, Ovarian cancer","Arcus Biosciences, Arcus Biosciences","multicentre, open, prospective, sequential",Number of Participants with Treatment Emergent...,...,Recruiting,2019-03-06|Other trial event|Last checked agai...,Advanced breast cancer|treatment|second-line o...,NaN,NaN,NaN,NaN,NaN,AB-928,"[30, Jul, 2021, (planned)]"
1,700240183,"A Phase 1b, Open-label Study to Assess the Saf...","Antibody-dependent cell cytotoxicity, Antimeta...","Antineoplastics, Antineoplastics, Antineoplastics","2 ring heterocyclic compounds, Diamines, Furan...",Monoclonal antibodies,"Advanced breast cancer, Male breast cancer","Cascadian Therapeutics, Cascadian Therapeutics...","multicentre, open, parallel, prospective",Incidence of adverse events\ntime_frame: Up to...,...,"Active, no longer recruiting",2019-03-06|Other trial event|Last checked agai...,Advanced breast cancer|treatment|second-line o...,HER2,HER2,NaN,NaN,NaN,Tucatinib,"[01, Jul, 2019, (planned)]"
2,700303393,"A Multicenter, Open-label Phase 2 Study of Len...",Platelet derived growth factor alpha receptor ...,"Antineoplastics, Immunotherapies, Antineoplastics","Amides, Chlorobenzenes, Cyclopropanes, Phenyl ...",Monoclonal antibodies,"Biliary cancer, Breast cancer, Colorectal canc...","Eisai Inc, Merck Sharp & Dohme, Merck Sharp & ...","multicentre, open, prospective",Objective Response Rate (ORR) per Response Eva...,...,Recruiting,2019-03-06|Other trial event|Last checked agai...,Biliary cancer|treatment|; Breast cancer|treat...,NaN,NaN,NaN,NaN,NaN,"Lenvatinib, Pembrolizumab","[11, Apr, 2022, (planned)]"
3,700227141,Cohort Study of Prospective Validation of Pred...,"Angiogenesis inhibitors, Angiogenesis inhibito...","Antineoplastics, Eye disorder therapies, Antin...","Diterpenes, Paclitaxels, Small molecules, Taxanes",Monoclonal antibodies,"Adenocarcinoma, Advanced breast cancer",NaN,"cohort, multicentre, observational, prospective",Measure of the initial rates and changes in CE...,...,"Active, no longer recruiting",2019-03-06|Other trial event|Last checked agai...,"Adenocarcinoma|treatment|first-line therapy, s...",NaN,NaN,NaN,NaN,NaN,"Bevacizumab, Paclitaxel","[01, Dec, 2020, (planned)]"
4,700243825,A Phase Ib/II Randomized Study of BI 836845 in...,"Aromatase inhibitors, B cell inhibitors, Estro...","Antineoplastics, Antineoplastics, Antineoplastics","Androstadienes, Small molecules, Macrolides, P...",Monoclonal antibodies,Advanced breast cancer,"Boehringer Ingelheim, Boehringer Ingelheim","multicentre, open, parallel, prospective, rand...",Progression-free survival (PFS)\n[ Time Frame:...,...,"Active, no longer recruiting",2019-03-06|Other trial event|Last checked agai...,Advanced breast cancer|treatment|second-line o...,NaN,NaN,NaN,NaN,"A1C, ER alpha, HER2, PGR",Xentuzumab,"[12, Nov, 2019, (planned)]"


In [31]:
trials['Indication'].head(10)

0               Advanced breast cancer, Ovarian cancer
1           Advanced breast cancer, Male breast cancer
2    Biliary cancer, Breast cancer, Colorectal canc...
3               Adenocarcinoma, Advanced breast cancer
4                               Advanced breast cancer
5                  Adenocarcinoma, Early breast cancer
6    Advanced breast cancer, Non-Hodgkin's lymphoma...
7    Advanced breast cancer, Cervical cancer, Endom...
8               Adenocarcinoma, Advanced breast cancer
9                    Breast cancer, Male breast cancer
Name: Indication, dtype: object

In [32]:
trials.columns

Index(['Trial ID', 'Trial Title', 'Mechanism of action',
       'Drug class (therapeutic effect)', 'Drug class (chemical)',
       'Drug class (biological)', 'Indication', 'Organisations',
       'Trial Design', 'Trial Endpoints', 'Primary Endpoints', 'Location',
       'Phase of Trial', 'Subject Age', 'Patient Exclusion Criteria',
       'Patient Inclusion Criteria', 'Subject Number', 'Subject Type',
       'Planned Subject Number', 'Trial Acronym(s)', 'Trial Identifier',
       'Drug Details', 'Trial Results', 'Trial Results - efficacy',
       'Trial Results - tolerability', 'Trial  Results - other',
       'Trial Centre Details', 'Lead Centre', 'Investigators',
       'Trial Comments', 'Trial Initiation date', 'Trial End date',
       'Trial Status', 'Trial History', 'Diseases treated',
       'Biomarkers-Inclusion', 'Biomarkers-Exclusion',
       'Biomarkers-Outcome Measure', 'Biomarkers-Safety',
       'Biomarkers-not specified', 'Primary Drugs'],
      dtype='object')

In [161]:
trials['Trial Initiation date']

0        15 Oct 2018 (actual)
1        01 Dec 2013 (actual)
2        12 Feb 2019 (actual)
3        01 Jun 2012 (actual)
4        15 May 2014 (actual)
5        27 Dec 2018 (actual)
6        16 Mar 2011 (actual)
7        12 Aug 2015 (actual)
8        22 Mar 2017 (actual)
9        22 Aug 2018 (actual)
10       01 Dec 2015 (actual)
11       05 Feb 2015 (actual)
12       13 Jan 2017 (actual)
13      01 Mar 2019 (planned)
14      28 Mar 2019 (planned)
15       15 Nov 2016 (actual)
16       12 Feb 2019 (actual)
17       01 Mar 2015 (actual)
18       05 Feb 2019 (actual)
19       01 Dec 2007 (actual)
20       22 Jun 2010 (actual)
21       01 Dec 2005 (actual)
22       16 Oct 2008 (actual)
23       20 Jun 2018 (actual)
24       24 Aug 2011 (actual)
25       06 Dec 2016 (actual)
26       17 Jan 2012 (actual)
27       01 Jul 2015 (actual)
28       01 Jan 2014 (actual)
29       14 Aug 2017 (actual)
                ...          
4499                (planned)
4500                (planned)
4501     2

In [71]:
trials['Drug Details'].values[2:5] #drug, method of use, trials

array(['Lenvatinib|Oral|Capsule; Pembrolizumab|Intravenous|Infusion',
       'Bevacizumab|Intravenous|Infusion; Paclitaxel||',
       'Everolimus|Oral|Tablet; Exemestane|Oral|; Xentuzumab|Intravenous|Infusion'],
      dtype=object)

In [ ]:
#str.split(trials['Trial History'].values[1], ';')

In [154]:
rows_trial_1 = str.split(trials['Trial History'].values[1], ';')
#columns_trial_1 = [str.split(i, '|') for i in rows_trial_1]
columns_trial_1 = str.split(rows_trial_1[0], '|')
rows_trial_1

['2019-03-06|Other trial event|Last checked against ClinicalTrials.gov record.',
 ' 2018-08-31|Completion date|Planned End Date changed from 1 Jul 2018 to 1 Jul 2019.',
 ' 2018-08-31|Other trial event|Planned primary completion date changed from 1 Jul 2018 to 1 Jul 2019.',
 ' 2018-08-31|Biomarker Update|Biomarkers information updated',
 ' 2018-06-05|Results|Results of an retrospective pooled analysis (NCT01983501 and NCT02025192) assessing clinical benefit of tucatinib after isolated brain progression presented at the 54th Annual Meeting of the American Society of Clinical Oncology',
 ' 2018-05-24|Results|Results assessing the safety, tolerabilty, pharmacokinetics and recommended phase 2 dose published in the Lancet Oncology',
 ' 2018-03-09|Other trial event|According to a Seattle Genetics media release, Cascadian Therapeutics has been acquired by Seattle Genetics, and this acquisition provides the Global Rights of Tucatinib to Seattle Genetics.',
 ' 2018-01-04|Other trial event|Accord

In [61]:
# one_split = trials['split_end_date'].values[0]
# pd.to_datetime(' '.join(one_split[:3]))

In [162]:
trials['split_start_date'] = trials['Trial Initiation date'].apply(lambda x: x.split(' '))

trials['split_end_date'] = trials['Trial End date'].apply(lambda x: x.split(' '))
in_progress_trials = trials[trials['split_start_date'].apply(lambda x: len(x[:3]))==3]
planned_trials = trials[trials['split_start_date'].apply(lambda x: len(x[:3]))==2]

In [111]:
planned_trials.shape[0]+in_progress_trials.shape[0]==trials.shape[0]

True

In [157]:
in_progress_trials.head()

,Trial ID,Trial Title,Mechanism of action,Drug class (therapeutic effect),Drug class (chemical),Drug class (biological),Indication,Organisations,Trial Design,Trial Endpoints,...,Trial Status,Trial History,Diseases treated,Biomarkers-Inclusion,Biomarkers-Exclusion,Biomarkers-Outcome Measure,Biomarkers-Safety,Biomarkers-not specified,Primary Drugs,split_end_date
0,700301099,A Phase 1/1b Study to Evaluate the Safety and ...,"Adenosine A2A receptor antagonists, Adenosine ...","Antineoplastics, Antineoplastics, Antivirals, ...","Small molecules, Anthracyclines, Doxorubicins,...",NaN,"Advanced breast cancer, Ovarian cancer","Arcus Biosciences, Arcus Biosciences","multicentre, open, prospective, sequential",Number of Participants with Treatment Emergent...,...,Recruiting,2019-03-06|Other trial event|Last checked agai...,Advanced breast cancer|treatment|second-line o...,NaN,NaN,NaN,NaN,NaN,AB-928,"[30, Jul, 2021, (planned)]"
1,700240183,"A Phase 1b, Open-label Study to Assess the Saf...","Antibody-dependent cell cytotoxicity, Antimeta...","Antineoplastics, Antineoplastics, Antineoplastics","2 ring heterocyclic compounds, Diamines, Furan...",Monoclonal antibodies,"Advanced breast cancer, Male breast cancer","Cascadian Therapeutics, Cascadian Therapeutics...","multicentre, open, parallel, prospective",Incidence of adverse events\ntime_frame: Up to...,...,"Active, no longer recruiting",2019-03-06|Other trial event|Last checked agai...,Advanced breast cancer|treatment|second-line o...,HER2,HER2,NaN,NaN,NaN,Tucatinib,"[01, Jul, 2019, (planned)]"
2,700303393,"A Multicenter, Open-label Phase 2 Study of Len...",Platelet derived growth factor alpha receptor ...,"Antineoplastics, Immunotherapies, Antineoplastics","Amides, Chlorobenzenes, Cyclopropanes, Phenyl ...",Monoclonal antibodies,"Biliary cancer, Breast cancer, Colorectal canc...","Eisai Inc, Merck Sharp & Dohme, Merck Sharp & ...","multicentre, open, prospective",Objective Response Rate (ORR) per Response Eva...,...,Recruiting,2019-03-06|Other trial event|Last checked agai...,Biliary cancer|treatment|; Breast cancer|treat...,NaN,NaN,NaN,NaN,NaN,"Lenvatinib, Pembrolizumab","[11, Apr, 2022, (planned)]"
3,700227141,Cohort Study of Prospective Validation of Pred...,"Angiogenesis inhibitors, Angiogenesis inhibito...","Antineoplastics, Eye disorder therapies, Antin...","Diterpenes, Paclitaxels, Small molecules, Taxanes",Monoclonal antibodies,"Adenocarcinoma, Advanced breast cancer",NaN,"cohort, multicentre, observational, prospective",Measure of the initial rates and changes in CE...,...,"Active, no longer recruiting",2019-03-06|Other trial event|Last checked agai...,"Adenocarcinoma|treatment|first-line therapy, s...",NaN,NaN,NaN,NaN,NaN,"Bevacizumab, Paclitaxel","[01, Dec, 2020, (planned)]"
4,700243825,A Phase Ib/II Randomized Study of BI 836845 in...,"Aromatase inhibitors, B cell inhibitors, Estro...","Antineoplastics, Antineoplastics, Antineoplastics","Androstadienes, Small molecules, Macrolides, P...",Monoclonal antibodies,Advanced breast cancer,"Boehringer Ingelheim, Boehringer Ingelheim","multicentre, open, parallel, prospective, rand...",Progression-free survival (PFS)\n[ Time Frame:...,...,"Active, no longer recruiting",2019-03-06|Other trial event|Last checked agai...,Advanced breast cancer|treatment|second-line o...,NaN,NaN,NaN,NaN,"A1C, ER alpha, HER2, PGR",Xentuzumab,"[12, Nov, 2019, (planned)]"


In [152]:
planned_trials['Trial History'].values[0]

'2019-03-06|Other trial event|Last checked against Clinicaltrials.gov record.; 2018-11-16|Results|Results presented at the 30th EORTC-NCI-AACR Symposium on Molecular Targets and Cancer Therapeutics; 2018-10-23|Results|Results (n=59) from patients with PTEN-deficient cancers enrolled to Arms N and P treated with GSK2636771, presented at the 43rd European Society for Medical Oncology Congress.; 2018-09-24|Other trial event|According to a NeoGenomics media release, the company announced today that it is participating as a designated laboratory in the NCI-Molecular Analysis for Therapy Choice (NCI-MATCH or EAY131) precision medicine cancer treatment clinical trial.; 2018-08-28|Other trial event|According to a OPKO Health media release, the BioReference Laboratories ( a subsidiary of OPKO Health ) has signed an agreement with NCI and ECOG-ACRIN to participate in this study.; 2018-08-02|Other trial event|According to a Tempus media release, the company (technology) is participating as a CLIA

In [ ]:
drugs[drugs['Drug Name'].isin(trials['Primary Drugs'])]

In [128]:
basic_trials_series = trials['Phase of Trial'].value_counts()

In [136]:
basic_trials_series

Phase II                  1837
Phase I                   1036
Phase III                  592
Phase IV                   503
Phase I/II                 452
Phase II/III                41
Phase 0                     35
Clinical Phase Unknown      32
Phase III/IV                 1
Name: Phase of Trial, dtype: int64

In [134]:
basic_trials_series.sum()

4529

In [38]:
trials['Organisations'].value_counts()

Roche, Roche                                                                                                              122
Pfizer                                                                                                                     96
AstraZeneca, AstraZeneca                                                                                                   93
Novartis                                                                                                                   90
Pfizer, Pfizer                                                                                                             80
AstraZeneca                                                                                                                75
Roche                                                                                                                      72
Genentech                                                                                                             

In [ ]:
#total number of trials = 4529
#total number that have been at phase 1

,Trial ID,Trial Title,Mechanism of action,Drug class (therapeutic effect),Drug class (chemical),Drug class (biological),Indication,Organisations,Trial Design,Trial Endpoints,...,Trial End date,Trial Status,Trial History,Diseases treated,Biomarkers-Inclusion,Biomarkers-Exclusion,Biomarkers-Outcome Measure,Biomarkers-Safety,Biomarkers-not specified,Primary Drugs
1,700240183,"A Phase 1b, Open-label Study to Assess the Saf...","Antibody-dependent cell cytotoxicity, Antimeta...","Antineoplastics, Antineoplastics, Antineoplastics","2 ring heterocyclic compounds, Diamines, Furan...",Monoclonal antibodies,"Advanced breast cancer, Male breast cancer","Cascadian Therapeutics, Cascadian Therapeutics...","multicentre, open, parallel, prospective",Incidence of adverse events\ntime_frame: Up to...,...,01 Jul 2019 (planned),"Active, no longer recruiting",2019-03-06|Other trial event|Last checked agai...,Advanced breast cancer|treatment|second-line o...,HER2,HER2,NaN,NaN,NaN,Tucatinib
10,700257487,"Phase 2 Randomized, Double-Blinded, Controlled...","Antibody-dependent cell cytotoxicity, Antibody...","Antineoplastics, Antineoplastics, Antineoplast...","2 ring heterocyclic compounds, Diamines, Furan...","Monoclonal antibodies, Monoclonal antibodies","Advanced breast cancer, Carcinoma, Male breast...","Cascadian Therapeutics, Cascadian Therapeutics...","double-blind, multicentre, parallel, prospecti...",Progression-free survival (PFS) per RECIST 1.1...,...,01 Jul 2021 (planned),Recruiting,2019-03-06|Other trial event|Last checked agai...,Advanced breast cancer|treatment|second-line o...,HER2,HER2,NaN,NaN,NaN,Tucatinib
16,700304216,"A Phase 1, Open-Label, Fixed-Sequence, Drug-Dr...","AMP activated protein kinase stimulants, Emiss...","Antineoplastics, Antihyperglycaemics","2 ring heterocyclic compounds, Diamines, Furan...",NaN,"Breast cancer, Colorectal cancer","Seattle Genetics, Seattle Genetics","open, prospective",Maximum observed plasma concentration (Cmax) o...,...,01 Mar 2019 (planned),Recruiting,2019-03-06|Other trial event|Last checked agai...,Breast cancer|treatment|; Colorectal cancer|tr...,NaN,NaN,NaN,NaN,NaN,Tucatinib
478,700302859,"A Phase 1, Randomized, Partially Double-Blind,...","DNA gyrase inhibitors, ERBB 2 receptor antagon...","Antineoplastics, Antibacterials","2 ring heterocyclic compounds, Diamines, Furan...",NaN,"Breast cancer, Colorectal cancer","Seattle Genetics, Seattle Genetics","crossover, double-blind, prospective, randomised",Placebo-corrected change-from-baseline in QTcF...,...,01 Mar 2019 (planned),Recruiting,2019-02-03|Other trial event|Last checked agai...,Breast cancer|treatment|; Colorectal cancer|tr...,NaN,NaN,NaN,NaN,NaN,Tucatinib
481,700238859,"A Phase 1b, Open-label Study to Assess the Saf...","Apoptosis stimulants, ERBB 2 receptor antagoni...","Antineoplastics, Antineoplastics","2 ring heterocyclic compounds, Diamines, Furan...","Immunoconjugates, Immunotoxins, Monoclonal ant...","Advanced breast cancer, Male breast cancer","Cascadian Therapeutics, Cascadian Therapeutics...","multicentre, open, prospective",Incidence and severity of adverse events\nsafe...,...,01 Jul 2019 (planned),"Active, no longer recruiting",2019-02-01|Other trial event|Last checked agai...,Advanced breast cancer|treatment|stage IV; Mal...,HER2,HER2,NaN,NaN,NaN,Tucatinib
489,700301177,"An Open-label, Non-randomized, Single-dose, Pa...",ERBB 2 receptor antagonists,Antineoplastics,"2 ring heterocyclic compounds, Diamines, Furan...",NaN,"Breast cancer, Colorectal cancer","Seattle Genetics, Seattle Genetics","multicentre, open, parallel, prospective",Maximum observed concentration\ndescription: P...,...,01 May 2019 (planned),Recruiting,2019-02-01|Other trial event|Last checked agai...,Breast cancer|treatment|; Colorectal cancer|tr...,NaN,NaN,NaN,NaN,NaN,Tucatinib
570,700301239,"A Phase 1, Open-Label, Fixed-sequence, 5-part,...","14-alpha demethylase inhibitors, Angiogenesis ...","Antineoplastics, Antihyperlipidaemics, Antifun...","2 ring he

In [ ]:
#Phase I 